Проанализируем файлы с исходными данными, определим, какие трансформации в ETL-процессе необходимо будет произвести

Описание файлов с данными:
* ft_balance_f – остатки средств на счетах
* ft_posting_f – проводки (движения средств) по счетам
* md_account_d – информация о счетах клиентов
* md_currency_d – справочник валют
* md_exchange_rate_d – курсы валют
* md_ledger_account_s – справочник балансовых счетов

In [1]:
import pandas as pd

IMPORT_PATH = '/home/joann/etl'

EXPORT_PATH = '/home/joann/etl/dag_src'

In [2]:
def read(filename, path, keys):
    try:
        df = pd.read_csv(f'{path}/{filename}', sep=';')
    except:
        df = pd.read_csv(f'{path}/{filename}', sep=';', encoding='CP866')
    display(df.head())
    display(df.info())
    
    print('\nВсего записей:', df.shape[0])
    for key in keys:
        print(key, '- количество уникальных:', df[key].value_counts().count())
    print('Количество дубликатов:', df.iloc[:,1:].duplicated().sum())

```sql
create table DS.FT_BALANCE_F (
  on_date DATE not null,
  account_rk NUMERIC not null,
  currency_rk NUMERIC,
  balance_out FLOAT,
  
  primary key (ON_DATE, ACCOUNT_RK)
);
```

In [3]:
FILENAME = 'ft_balance_f.csv'
KEYS = ['ON_DATE', 'ACCOUNT_RK']

read(FILENAME, IMPORT_PATH, KEYS)

,\t,ON_DATE,ACCOUNT_RK,CURRENCY_RK,BALANCE_OUT
0,1,31.12.2017,36237725,35,0.00
1,2,31.12.2017,24656,35,80533.62
2,3,31.12.2017,18849846,34,63891.96
3,4,31.12.2017,1972647,34,87732.10
4,5,31.12.2017,34157174,34,97806.90


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   	            112 non-null    int64  
 1   ON_DATE      112 non-null    object 
 2   ACCOUNT_RK   112 non-null    int64  
 3   CURRENCY_RK  112 non-null    int64  
 4   BALANCE_OUT  112 non-null    float64
dtypes: float64(1), int64(3), object(1)
memory usage: 4.5+ KB


None


Всего записей: 112
ON_DATE - количество уникальных: 1
ACCOUNT_RK - количество уникальных: 112
Количество дубликатов: 0


В файле `ft_balance_f.csv` необходимо:
1. Удалить первый столбец
2. Преобразовать тип данных колонки `ON_DATE` в `datetime`

```sql
create table DS.FT_POSTING_F (
  oper_date DATE not null,
  credit_account_rk NUMERIC not null,
  debet_account_rk NUMERIC not null,
  credit_amount FLOAT,
  debet_amount FLOAT,
  
  primary key (OPER_DATE, CREDIT_ACCOUNT_RK, DEBET_ACCOUNT_RK)
);
```

In [4]:
FILENAME = 'ft_posting_f.csv'

KEYS = ['OPER_DATE', 'CREDIT_ACCOUNT_RK', 'DEBET_ACCOUNT_RK']

read(FILENAME, IMPORT_PATH, KEYS)

,Unnamed: 0,OPER_DATE,CREDIT_ACCOUNT_RK,DEBET_ACCOUNT_RK,CREDIT_AMOUNT,DEBET_AMOUNT
0,1,2018-01-09,13630,17436,94333.93,18337.76
1,2,2018-01-09,15698716,13630,68294.14,31542.06
2,3,2018-01-09,12048338,13630,2192.96,98734.33
3,4,2018-01-09,393808409,17244,44179.86,98544.65
4,5,2018-01-09,409685020,13630,18843.05,889.74


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33892 entries, 0 to 33891
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         33892 non-null  int64  
 1   OPER_DATE          33892 non-null  object 
 2   CREDIT_ACCOUNT_RK  33892 non-null  int64  
 3   DEBET_ACCOUNT_RK   33892 non-null  int64  
 4   CREDIT_AMOUNT      33892 non-null  float64
 5   DEBET_AMOUNT       33892 non-null  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.6+ MB


None


Всего записей: 33892
OPER_DATE - количество уникальных: 17
CREDIT_ACCOUNT_RK - количество уникальных: 477
DEBET_ACCOUNT_RK - количество уникальных: 731
Количество дубликатов: 0


В файле `ft_posting.csv` необходимо:
1. Удалить первый столбец
2. Преобразовать тип данных колонки `OPER_DATE` в `datetime`
3. Сгруппировать данные по столбцам, являющимся первичными ключами

```sql
create table DS.MD_ACCOUNT_D (
  data_actual_date DATE not null,
  data_actual_end_date DATE not null,
  account_rk NUMERIC not null,
  account_number VARCHAR(20) not null,
  char_type VARCHAR(1) not null,
  currency_rk NUMERIC not null,
  currency_code VARCHAR(3) not null,
  
  primary key (DATA_ACTUAL_DATE, ACCOUNT_RK)
);
```

In [5]:
FILENAME = 'md_account_d.csv'
KEYS = ['DATA_ACTUAL_DATE', 'ACCOUNT_RK']

read(FILENAME, IMPORT_PATH, KEYS)

,,DATA_ACTUAL_DATE,DATA_ACTUAL_END_DATE,ACCOUNT_RK,ACCOUNT_NUMBER,CHAR_TYPE,CURRENCY_RK,CURRENCY_CODE
0,1,2018-01-01,2018-01-31,36237725,30425840700000583001,A,35,840
1,2,2018-01-01,2018-01-31,24656,30114840700000770002,A,35,840
2,3,2018-01-01,2018-01-31,18849846,30109810500000435003,P,34,643
3,4,2018-01-01,2018-01-31,1972647,30111810700000908001,P,34,643
4,5,2018-01-01,2018-01-31,34157174,30424810100000583001,A,34,643


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0                         112 non-null    int64 
 1   DATA_ACTUAL_DATE      112 non-null    object
 2   DATA_ACTUAL_END_DATE  112 non-null    object
 3   ACCOUNT_RK            112 non-null    int64 
 4   ACCOUNT_NUMBER        112 non-null    object
 5   CHAR_TYPE             112 non-null    object
 6   CURRENCY_RK           112 non-null    int64 
 7   CURRENCY_CODE         112 non-null    int64 
dtypes: int64(4), object(4)
memory usage: 7.1+ KB


None


Всего записей: 112
DATA_ACTUAL_DATE - количество уникальных: 1
ACCOUNT_RK - количество уникальных: 112
Количество дубликатов: 0


В файле `md_account_d.csv` необходимо:
1. Удалить первый столбец
2. Преобразовать тип данных колонок `DATA_ACTUAL_DATE` и `DATA_ACTUAL_END_DATE` в `datetime`

```sql
create table DS.MD_CURRENCY_D (
  currency_rk NUMERIC not null,
  data_actual_date DATE not null,
  data_actual_end_date DATE,
  currency_code VARCHAR(3),
  code_iso_char VARCHAR(3),
  
  primary key (CURRENCY_RK, DATA_ACTUAL_DATE)
);
```

In [6]:
FILENAME = 'md_currency_d.csv'
KEYS = ['CURRENCY_RK', 'DATA_ACTUAL_DATE']

read(FILENAME, IMPORT_PATH, KEYS)

,,CURRENCY_RK,DATA_ACTUAL_DATE,DATA_ACTUAL_END_DATE,CURRENCY_CODE,CODE_ISO_CHAR
0,1,4586704,2011-09-06,2050-12-31,000,NON
1,2,50,2017-05-11,2050-12-31,356,INR
2,3,51,2017-05-11,2050-12-31,484,MXN
3,4,52,2017-05-11,2050-12-31,434,LYD
4,5,53,2017-05-11,2050-12-31,422,LBR


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0                         50 non-null     int64 
 1   CURRENCY_RK           50 non-null     int64 
 2   DATA_ACTUAL_DATE      50 non-null     object
 3   DATA_ACTUAL_END_DATE  50 non-null     object
 4   CURRENCY_CODE         50 non-null     object
 5   CODE_ISO_CHAR         50 non-null     object
dtypes: int64(2), object(4)
memory usage: 2.5+ KB


None


Всего записей: 50
CURRENCY_RK - количество уникальных: 50
DATA_ACTUAL_DATE - количество уникальных: 3
Количество дубликатов: 0


В файле `md_currency_d.csv` необходимо:
1. Удалить первый столбец
2. Преобразовать тип данных колонок `DATA_ACTUAL_DATE` и `DATA_ACTUAL_END_DATE` в `datetime`
3. Использовать подходящую кодировку `CP866`

```sql
create table DS.MD_EXCHANGE_RATE_D (
  data_actual_date DATE not null,
  data_actual_end_date DATE,
  currency_rk NUMERIC not null,
  reduced_cource FLOAT,
  code_iso_num VARCHAR(3),
  
  primary key (DATA_ACTUAL_DATE, CURRENCY_RK)
);
```

In [7]:
FILENAME = 'md_exchange_rate_d.csv'

KEYS = ['DATA_ACTUAL_DATE', 'CURRENCY_RK']

read(FILENAME, IMPORT_PATH, KEYS)

,,DATA_ACTUAL_DATE,DATA_ACTUAL_END_DATE,CURRENCY_RK,REDUCED_COURCE,CODE_ISO_NUM
0,1,2018-01-31,2018-01-31,427870281,28.37980,933
1,2,2018-01-31,2018-01-31,29,7.11613,752
2,3,2018-01-31,2018-01-31,529511970,33.05430,944
3,4,2018-01-31,2018-01-31,62,8.87555,156
4,5,2018-01-31,2018-01-31,205699733,2.74512,203


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9614 entries, 0 to 9613
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0                         9614 non-null   int64  
 1   DATA_ACTUAL_DATE      9614 non-null   object 
 2   DATA_ACTUAL_END_DATE  9614 non-null   object 
 3   CURRENCY_RK           9614 non-null   int64  
 4   REDUCED_COURCE        9614 non-null   float64
 5   CODE_ISO_NUM          9614 non-null   int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 450.8+ KB


None


Всего записей: 9614
DATA_ACTUAL_DATE - количество уникальных: 249
CURRENCY_RK - количество уникальных: 28
Количество дубликатов: 3395


В файле `md_exchange_rate_d.csv` необходимо:
1. Удалить первый столбец
2. Преобразовать тип данных колонок `DATA_ACTUAL_DATE` и `DATA_ACTUAL_END_DATE` в `datetime`
3. Удалить дубликаты

```sql
create table DS.MD_LEDGER_ACCOUNT_S (
  chapter CHAR(1),
  chapter_name VARCHAR(16),
  section_number INTEGER,
  section_name VARCHAR(22),
  subsection_name VARCHAR(21),
  ledger1_account INTEGER,
  ledger1_account_name VARCHAR(47),
  ledger_account INTEGER not null,
  ledger_account_name VARCHAR(153),
  characteristic CHAR(1),
  is_resident INTEGER,
  is_reserve INTEGER,
  is_reserved INTEGER,
  is_loan INTEGER,
  is_reserved_assets INTEGER,
  is_overdue INTEGER,
  is_interest INTEGER,
  pair_account VARCHAR(5),
  start_date DATE not null,
  end_date DATE,
  is_rub_only INTEGER,
  min_term VARCHAR(1),
  min_term_measure VARCHAR(1),
  max_term VARCHAR(1),
  max_term_measure VARCHAR(1),
  ledger_acc_full_name_translit VARCHAR(1),
  is_revaluation VARCHAR(1),
  is_correct VARCHAR(1),
  
  primary key (LEDGER_ACCOUNT, START_DATE)
);
```

In [8]:
FILENAME = 'md_ledger_account_s.csv'
KEYS = ['LEDGER_ACCOUNT', 'START_DATE']

read(FILENAME, IMPORT_PATH, KEYS)

,,CHAPTER,CHAPTER_NAME,SECTION_NUMBER,SECTION_NAME,SUBSECTION_NAME,LEDGER1_ACCOUNT,LEDGER1_ACCOUNT_NAME,LEDGER_ACCOUNT,LEDGER_ACCOUNT_NAME,...,START_DATE,END_DATE,IS_RUB_ONLY,MIN_TERM,MIN_TERM_MEASURE,MAX_TERM,MAX_TERM_MEASURE,LEDGER_ACC_FULL_NAME_TRANSLIT,IS_REVALUATION,IS_CORRECT
0,1,А,Балансовые счета,3,МЕЖБАНКОВСКИЕ ОПЕРАЦИИ,МЕЖБАНКОВСКИЕ РАСЧЕТЫ,302,Счета кредитных организаций по другим операциям,30204,Обязательные резервы кредитных организаций по ...,...,2014-01-01,2050-12-31,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,А,Балансовые счета,3,МЕЖБАНКОВСКИЕ ОПЕРАЦИИ,МЕЖБАНКОВСКИЕ РАСЧЕТЫ,301,Корреспондентские счета,30109,Корреспондентские счета кредитных организаций ...,...,2008-01-01,2050-12-31,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,А,Балансовые счета,3,МЕЖБАНКОВСКИЕ ОПЕРАЦИИ,МЕЖБАНКОВСКИЕ РАСЧЕТЫ,301,Корреспондентские счета,30110,Корреспондентские счета в кредитных организаци...,...,2008-01-01,2050-12-31,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,А,Балансовые счета,3,МЕЖБАНКОВСКИЕ ОПЕРАЦИИ,МЕЖБАНКОВСКИЕ РАСЧЕТЫ,301,Корреспондентские счета,30111,Корреспондентские счета банков- нерезидентов,...,2008-01-01,2050-12-31,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,А,Балансовые счета,3,МЕЖБАНКОВСКИЕ ОПЕРАЦИИ,МЕЖБАНКОВСКИЕ РАСЧЕТЫ,301,Корреспондентские счета,30126,Резервы на возможные потери,...,2008-01-01,2050-12-31,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0                                  18 non-null     int64  
 1   CHAPTER                        18 non-null     object 
 2   CHAPTER_NAME                   18 non-null     object 
 3   SECTION_NUMBER                 18 non-null     int64  
 4   SECTION_NAME                   18 non-null     object 
 5   SUBSECTION_NAME                18 non-null     object 
 6   LEDGER1_ACCOUNT                18 non-null     int64  
 7   LEDGER1_ACCOUNT_NAME           18 non-null     object 
 8   LEDGER_ACCOUNT                 18 non-null     int64  
 9   LEDGER_ACCOUNT_NAME            18 non-null     object 
 10  CHARACTERISTIC                 18 non-null     object 
 11  IS_RESIDENT                    18 non-null     int64  
 12  IS_RESERVE                     18 non-null     int64

None


Всего записей: 18
LEDGER_ACCOUNT - количество уникальных: 18
START_DATE - количество уникальных: 4
Количество дубликатов: 0


В файле `md_ledger_account_s.csv` необходимо:
1. Удалить первый столбец
2. Преобразовать тип данных колонок `START_DATE` и `END_DATE` в `datetime`
3. Использовать подходящую кодировку `CP866`

В результате анализа исходных данных установлено:
1. Удаления первого столбца требуют все файлы
2. Все файлы имеют разделитель `;`
3. У всех файлов, кроме двух, кодировка стандартная `utf8`, у `md_currency_d.csv` и `md_ledger_account_s.csv` - `CP866`
4. Все колонки с датами требуют преобразования типа данных в `datetime`, также все эти колонки содержат в названии слово DATE
5. В файле ft_posting_f.csv необходимо сгруппировать данные по столбцам, являющимся первичными ключами
6. В файле md_exchange_rate_d.csv необходимо удалить дубликаты